# Network Sniffing
Network sniffing refers to using the network interface on a system to monitor or capture information sent over a wired or wireless connection. An adversary may place a network interface into promiscuous mode to passively access data in transit over the network, or use span ports to capture a larger amount of data.

## Rule Content
```
- title: Network Sniffing
  id: ba1f7802-adc7-48b4-9ecb-81e227fddfd5
  status: experimental
  description: Network sniffing refers to using the network interface on a system
    to monitor or capture information sent over a wired or wireless connection. An
    adversary may place a network interface into promiscuous mode to passively access
    data in transit over the network, or use span ports to capture a larger amount
    of data.
  author: Timur Zinniatullin, oscd.community
  date: 2019/10/21
  modified: 2019/11/04
  references:
  - https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1040/T1040.yaml
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
    - Image|endswith: \tshark.exe
      CommandLine|contains: -i
    - Image|endswith: \windump.exe
    condition: selection
  falsepositives:
  - Admin activity
  fields:
  - Image
  - CommandLine
  - User
  - LogonGuid
  - Hashes
  - ParentProcessGuid
  - ParentCommandLine
  level: low
  tags:
  - attack.credential_access
  - attack.discovery
  - attack.t1040

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:*\\tshark.exe AND process_command_line.keyword:*\-i*) OR process_path.keyword:*\\windump.exe)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()